In [31]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import functions
import os


In [11]:
trips = pd.read_csv("../../../data/mmt_gtfs/trips.csv") # 11,640
stop_times = pd.read_csv("../../../data/mmt_gtfs/stop_times.csv") # 431,489
stops = pd.read_csv("../../../data/mmt_gtfs/stops.csv") # 2131

# route 2
route2_trips = trips[trips["route_short_name"] == 2]
route2_trips.head()

,route_id,route_short_name,service_id,trip_id,trip_headsign,direction_id,direction_name,block_id,shape_id,shape_code,trip_type,trip_sort,wheelchair_accessible,bikes_allowed
1,9028,2,92_WKD,1007843,NORTH TRANSFER: VIA SHERMAN,0,North Transfer,194263,55156,2S,W,20100,1,1
2,9028,2,92_WKD,1007844,NORTH TRANSFER: VIA FORDEM,0,North Transfer,194056,55154,2F,W,21600,1,1
3,9028,2,92_WKD,1007499,NORTH TRANSFER: VIA SHERMAN,0,North Transfer,194052,55156,2S,D,23400,1,1
4,9028,2,92_WKD,1007497,NORTH TRANSFER: VIA FORDEM,0,North Transfer,194058,55154,2F,D,25200,1,1
5,9028,2,92_WKD,1007481,NORTH TRANSFER: VIA SHERMAN,0,North Transfer,192750,55156,2S,D,27000,1,1


In [20]:
set(route2_trips["trip_headsign"])

{'CAPITOL SQUARE',
 'NORTH TRANSFER: VIA FORDEM',
 'NORTH TRANSFER: VIA SHERMAN',
 'WEST TRANSFER: VIA FORDEM',
 'WEST TRANSFER: VIA SHERMAN'}

In [24]:
route2_trips[route2_trips["trip_headsign"] == "CAPITOL SQUARE"]

,route_id,route_short_name,service_id,trip_id,trip_headsign,direction_id,direction_name,block_id,shape_id,shape_code,trip_type,trip_sort,wheelchair_accessible,bikes_allowed
36,9028,2,92_WKD,1007500,CAPITOL SQUARE,0,North Transfer,194055,55155,2Q,H,82800,1,1
1468,9028,2,92_SAT,1007568,CAPITOL SQUARE,0,North Transfer,194074,55155,2Q,H,82800,1,1
2453,9028,2,92_SUN,1007636,CAPITOL SQUARE,0,North Transfer,194088,55155,2Q,H,82800,1,1
4283,9028,2,92_MWK,1007772,CAPITOL SQUARE,0,North Transfer,194116,55155,2Q,H,82800,1,1
5892,9279,2,93_WKD,1019346,CAPITOL SQUARE,0,North Transfer,195020,56619,2Q,H,82800,1,1
7325,9279,2,93_SAT,1019418,CAPITOL SQUARE,0,North Transfer,195036,56619,2Q,H,82800,1,1
8310,9279,2,93_SUN,1019486,CAPITOL SQUARE,0,North Transfer,195050,56619,2Q,H,82800,1,1
10140,9279,2,93_MWK,1019612,CAPITOL SQUARE,0,North Transfer,195077,56619,2Q,H,82800,1,1


In [50]:
def write_directions(route_num):
    route_df = trips[trips["route_short_name"] == route_num]
    for service in list(set(route_df["service_id"])):
        if service[:2] != "92":
            continue
        service_df = route_df[route_df["service_id"] == service]
        for headsign in list(set(service_df["trip_headsign"])):
            trip_id = service_df[service_df["trip_headsign"] == headsign].iloc[0, 3]
            df = stop_times[stop_times["trip_id"] == trip_id][["stop_sequence", "stop_id",
                                                               "arrival_time", "departure_time",
                                                               "stop_headsign", "shape_dist_traveled"]]
            df = df.reset_index(drop=True)
            dirname = service
            if not os.path.exists(os.path.join("routes", dirname)):
                os.mkdir(os.path.join("routes", dirname))
            filename = "route" + str(route_num) + "_" +\
                       headsign.replace(" ", "_").replace(":", "").lower() + ".csv"
            filepath = os.path.join("routes", dirname, filename)
            df.to_csv(path_or_buf=filepath, index=False)

for route_num in list(functions.get_num_stops_of_route().keys()):
    write_directions(route_num)

In [48]:
def check_length(route_num):
    route_df = trips[trips["route_short_name"] == route_num]
    num_stops = []
    for service in list(set(route_df["service_id"])):
        by_service = []
        if service[:2] != "92":
            continue
        service_df = route_df[route_df["service_id"] == service]
        for headsign in list(set(service_df["trip_headsign"])):
            trip_id = service_df[service_df["trip_headsign"] == headsign].iloc[0, 3]
            df = stop_times[(stop_times["trip_id"] == trip_id)]
            by_service.append(len(df))
        num_stops.append(by_service)
    return num_stops

routes = dict()
for route_num in list(functions.get_num_stops_of_route().keys()):
    routes[route_num] = check_length(route_num)

routes

{2: [[57, 58, 58, 57, 37],
  [57, 58, 58, 57, 37],
  [57, 58, 58, 57, 37],
  [58, 57, 57, 58],
  [57, 58, 58, 57, 37]],
 4: [[59, 55], [59, 55], [59, 55], [59, 55], [59, 55]],
 5: [[19, 20, 46], [19, 20, 46], [19, 45, 46], [20, 46], [19, 20, 46]],
 6: [[82, 70, 34, 80, 70, 43, 42, 28, 78],
  [82, 70, 34, 80, 70, 43, 42, 28, 78],
  [87, 82, 70, 80, 70, 43, 28, 78],
  [82, 70, 34, 70, 42, 78],
  [82, 70, 34, 80, 70, 43, 42, 28, 78]],
 7: [[55, 56], [55, 56], [55, 56], [55, 56], [55, 56]],
 8: [[39, 30], [39, 30], [39, 30], [39, 30], [39, 30]],
 10: [[47, 51, 36, 35]],
 11: [[36, 55, 39, 58]],
 12: [[26, 38]],
 13: [[30, 28], [30, 28], [30, 28], [30, 28], [30, 28]],
 15: [[43, 58, 58]],
 16: [[20, 40, 20], [20, 40, 20], [20, 40, 20], [20, 40, 20], [20, 40, 20]],
 17: [[10, 10], [10, 10], [10, 10], [10, 10], [10, 10]],
 18: [[30, 37, 27, 40, 29],
  [30, 37, 27, 40, 29],
  [30, 37, 33, 40, 29, 32],
  [30, 37, 27, 40, 29],
  [30, 37, 27, 40, 29]],
 20: [[27, 28], [27, 28], [27, 28], [27, 28]